In [1]:
# Importing the necessary libraries
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import sklearn
import scipy as sp
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator

In [2]:
def readAsArray(img):
  band1 = img.GetRasterBand(1).ReadAsArray()
  return(band1)

In [3]:
def process_image(img):
    data = img.ReadAsArray()
    data = np.moveaxis(data, 0, -1)
    rows, cols, bands = data.shape
    data_2d = data.reshape(rows*cols, bands)
    data_2d_scaled = scaler.fit_transform(data_2d)
    data_2d_scaled = np.nan_to_num(data_2d_scaled)
    return data_2d_scaled

In [4]:
# Defining Plot Function

def plot_image(plt_img):
  plt.figure()
  plt.imshow(plt_img)
  plt.show()

In [5]:
# Running the DBSCAN Algorithm
first = True
first2 = True
scaler = StandardScaler()

for i in range(2,3):
    # List of images
    images_list=[str(i)+ "_P_256.tif", str(i) +"_A_256.tif"]
    images = []
    # Opening the satellite images
    for img_name in images_list:
        img = gdal.Open('./ALL_FLOOD_256/'+ img_name)
        data_2d_scaled = process_image(img)

        if(first):
            train_image=data_2d_scaled
            first = False
        else:
            train_image = np.concatenate([data_2d_scaled, train_image], axis=0)
    if (first2):
        combined_data = train_image
    else:
        combined_data = np.concatenate([combined_data, train_image], axis=0)

In [6]:
# Opening the test images
imgP = gdal.Open("./ALL_FLOOD_256/1_P_256.tif")
imgA = gdal.Open("./ALL_FLOOD_256/1_A_256.tif")

before_img = readAsArray(imgP)
after_img = readAsArray(imgA)

data_2d_scaled_before = process_image(imgP)
data_2d_scaled_after = process_image(imgA)

In [7]:
import csv
# headerList = ["Min Samples","Epsilon","No. of Before Clusters", "No. of After Clusters", "No. of Before Noise points", "No. of After Noise points"]

# with open('./myfile.csv', 'a', newline='') as f:
#     writer = csv.DictWriter(f,fieldnames=headerList)
#     writer.writeheader()
    
for k in range (50,60,10):
    nearest_neighbors = NearestNeighbors(n_neighbors=k)
    neighbors = nearest_neighbors.fit(combined_data)
    distances, indices = neighbors.kneighbors(combined_data)
    distances = np.sort(distances[:,k-1], axis=0)
    i = np.arange(len(distances))
    knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
    epsilon = distances[knee.knee]
    epsilon -=0.005
    max_epsilon = epsilon+0.011

    while (epsilon<=max_epsilon):

        dbscan_model = DBSCAN(eps=epsilon, min_samples = k)
        first = True
        first2 = True
        for i in range(2,3):
            # List of images
            images_list=[str(i)+ "_P_256.tif", str(i) +"_A_256.tif"]
            for img_name in images_list:
                img = gdal.Open('./ALL_FLOOD_256/'+ img_name)
                data_2d_scaled = process_image(img)
                if(first):
                    train_image=data_2d_scaled
                    first = False
                else:
                    train_image = np.concatenate([data_2d_scaled, train_image], axis=0)
            if (first2):
                combined_data = train_image
            else:
                combined_data = np.concatenate([combined_data, train_image], axis=0)
            dbscan_model.fit(combined_data)
            print(dbscan_model.labels_.shape)
        

        dbscan_before_labels = dbscan_model.fit_predict(data_2d_scaled_before)
        core_samples_mask_before = np.zeros_like(dbscan_before_labels, dtype=bool)
        core_samples_mask_before[dbscan_model.core_sample_indices_] = True
        labels_before = dbscan_before_labels
        n_clusters_before = len(set(labels_before)) - (1 if -1 in labels_before else 0)
        n_noise_before = list(labels_before).count(-1)

        dbscan_after_labels = dbscan_model.fit_predict(data_2d_scaled_after)
        core_samples_mask_after = np.zeros_like(dbscan_after_labels, dtype=bool)
        core_samples_mask_after[dbscan_model.core_sample_indices_] = True
        labels_after = dbscan_after_labels
        n_clusters_after = len(set(labels_after)) - (1 if -1 in labels_after else 0)
        n_noise_after = list(labels_after).count(-1)

        with open('tuning.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            print([k, epsilon, n_clusters_before, n_noise_before, n_clusters_after, n_noise_after])
            writer.writerow([k, epsilon, n_clusters_before, n_noise_before, n_clusters_after, n_noise_after])
        epsilon+=0.001

(131072,)
[50, 0.07639912863844894, 4, 10524, 5, 8267]
(131072,)
[50, 0.07739912863844894, 4, 10366, 5, 7990]
(131072,)
[50, 0.07839912863844894, 5, 10193, 6, 7807]
(131072,)
[50, 0.07939912863844895, 6, 10033, 4, 7519]
(131072,)
[50, 0.08039912863844895, 7, 9832, 4, 7326]
(131072,)
[50, 0.08139912863844895, 8, 9570, 2, 7030]
(131072,)
[50, 0.08239912863844895, 8, 9440, 2, 6829]
(131072,)
[50, 0.08339912863844895, 8, 9228, 1, 6570]
(131072,)
[50, 0.08439912863844895, 12, 8880, 3, 6336]
(131072,)
[50, 0.08539912863844895, 8, 8737, 4, 6196]
(131072,)
[50, 0.08639912863844895, 5, 8617, 5, 6027]


In [8]:
import sys
sys.argv = [{"test_var":"Testing","successful_execution":True}]
execfile("mailing.py")

Sending mail to: {'To': 'sparshtgupta@gmail.com', 'Cc': 'hardikriyasparsh@gmail.com', 'Bcc': ''}
Mail sent successfully! Sent at: 2023-04-18 09:04:10.330286
